<a href="https://colab.research.google.com/github/saneens/SpaceInvaderDoubleDQN/blob/main/DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install gym pyvirtualdisplay -qq
!pip install folium==0.2.1
!apt-get install -y xvfb python-opengl ffmpeg -qq

!apt-get update -qq
!apt-get install cmake -qq
!pip install --upgrade setuptools -qq
!pip install ez_setup -qq

     |████████████████████████████████| 71kB 9.4MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79979 sha256=8eca8cefce5d509d4d9cb94427cee2b3c79b6510fb4389b365a758345f901f74
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Selecting previously unselected package python-opengl.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.4_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.4) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Setting up xvfb (2:1.19.6-1ubuntu4.4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!pip install blosc

     |████████████████████████████████| 819kB 35.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for blosc: filename=blosc-1.9.1-cp36-cp36m-linux_x86_64.whl size=458896 sha256=e14f1b81546cd96b66f608660835bfad375153e1052d9f0d402c2a72500c9617
  Stored in directory: /root/.cache/pip/wheels/d5/c6/ad/3a62998b35309a61b84b306447c85bf639c5d539609c222c36
Successfully built blosc


In [ ]:
!pip install --upgrade wandb -qq

     |████████████████████████████████| 1.4MB 13.5MB/s 
     |████████████████████████████████| 112kB 52.0MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 102kB 14.3MB/s 
     |████████████████████████████████| 460kB 45.8MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 


In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(30)

import csv
import scipy.ndimage as ndimage
import bisect
import blosc
import pickle
import pandas as pd
import numpy as np
import random
import math
import glob
import io
import os
import cv2
import base64
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque
from datetime import datetime
import keras
from tensorflow.keras import datasets, layers, models
from keras import losses
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.models import load_model

from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# import wandb
import wandb


Using TensorFlow backend.


In [ ]:
#Class to manage the environment, makes it easier to control and perform actions on the environment.
#Invokes function to preprocess the image and stack four frames together
#Also takes care of the gym environment not displaying all the details in a frame
#Nomenclature
#step_number : Total steps performed by the environment, 4 actions are counted as a step. Same action is repeated four times everytime
#episode_step : Steps performed by the env. in the current game
#game_number : Number of game the environment is playing currently
class GameAgent:
    def __init__(self, args):
        self.env = gym.make("SpaceInvaders-v0")
        self.action_size = self.env.action_space.n
        self.game_number = 0
        self.step_number = 0
        self.epoch_number = 0
        self.game_done = True
        self.reset_game()

    def reset_game(self):
        if self.game_done:
            self.game_number += 1
        self.current_frame = self.env.reset()
        self.game_done = False
        self.lives_left = 3
        self.state = State().state_plus_frame(self.current_frame)
        self.game_score = 0
        self.episode_step = 0

    def take_action(self, action):
        reward = 0
        terminal = 0
        self.step_number += 1
        self.episode_step += 1

        for i in range(4):
            self.previous_frame = self.current_frame
            self.current_frame, tmp_rwd, self.game_done, life = self.env.step(action)
            reward += tmp_rwd

            # Detect end of episode by comparing the life remaining of the agent
            if life.get('ale.lives') < self.lives_left or self.game_done:
                terminal = 1
                self.lives_left = life.get('ale.lives')
                break

        #take the max of adjacent frames to avoid the gym rendering problem
        maxed_frame = np.maximum(self.previous_frame, self.current_frame)
        self.state = self.state.state_plus_frame(maxed_frame)
        self.game_score += reward
        return reward, self.state, terminal

In [ ]:
#Class to do the preprocessing and stacking of four images together
class State:
    def preprocess(self,frame):
        frame = frame[:, :, 0] * 0.2126 + frame[:, :, 1] * 0.0722 + frame[:, :, 2] * 0.7152
        frame = frame.astype(np.uint8)
        frame = cv2.resize(frame, (84, 84), interpolation=cv2.INTER_LINEAR)
        frame.resize((84, 84, 1))
        return frame

    def state_plus_frame(self, frame):
        frame = self.preprocess(frame)
        frame = blosc.compress(np.reshape(frame, 84 * 84).tobytes(), typesize=1)
        new_state = State()
        if hasattr(self, 'screens'):
            new_state.screens = self.screens[:3]
            new_state.screens.insert(0, frame)
        else:
            new_state.screens = [frame, frame, frame, frame]
        return new_state

    def get_screens(self):
        s = []
        for i in range(4):
            s.append(np.reshape(np.fromstring(blosc.decompress(self.screens[i]), dtype=np.uint8), (84, 84, 1)))
        s = np.concatenate(s, axis=2)
        s = np.reshape(s, (1, 84, 84, 4))
        return s

In [ ]:
class MemoryType:
    def __init__(self, current_state, action, reward, next_state, terminal):
        self.current_state = current_state
        self.action = action
        self.reward = reward
        self.next_state = next_state
        self.terminal = terminal
        self.weight = 1
        self.cumulative_weight = 1

    def isInteresting(self):
        return self.terminal or self.reward != 0

    def __lt__(self, obj):
        return self.cumulative_weight < obj.cumulative_weight


class StorageMemory:

    def __init__(self, args):
        self.memory = []
        self.max_capacity = args.replay_capacity
        self.prioritized_replay = args.prioritized_replay
        self.num_interesting_memory = 0
        self.batches_drawn = 0

    def add_memory(self, sample):
        self.memory.append(sample)
        self.update_weights()
        self.truncate_memory()

    def update_weights(self):
        if len(self.memory) > 1:
            self.memory[-1].cumulative_weight = self.memory[-1].weight + self.memory[-2].cumulative_weight

        if self.memory[-1].isInteresting():
            self.num_interesting_memory += 1

            # Boost the neighboring memory.  How many memory?  Roughly the number of memory
            uninteresting_range = int(max(1, len(self.memory) / max(1, self.num_interesting_memory)))
            for i in range(uninteresting_range, 0, -1):
                index = len(self.memory) - i
                if index < 1:
                    break
                # This is an exponential that ranges from 3.0 to 1.01 over the domain of [0, uninteresting_range]
                # So the interesting sample gets a 3x boost, and the one furthest away gets a 1% boost
                boost = 1.0 + 3.0/(math.exp(i/(uninteresting_range/6.0)))
                self.memory[index].weight *= boost
                self.memory[index].cumulative_weight = self.memory[index].weight + self.memory[index - 1].cumulative_weight

    def truncate_memory(self):
        if len(self.memory) > self.max_capacity * 1.05:
            truncated_weight = 0
            # Before truncating the list, correct self.num_interesting_memory, and prepare
            # for correcting the cumulativeWeights of the remaining memory
            for i in range(self.max_capacity, len(self.memory)):
                truncated_weight += self.memory[i].weight
                if self.memory[i].isInteresting():
                    self.num_interesting_memory -= 1

            # Truncate the list
            self.memory = self.memory[(len(self.memory) - self.max_capacity):]

            # Correct cumulativeWeights
            for sample in self.memory:
                sample.cumulative_weight -= truncated_weight

    def draw_batch(self, batch_size):
        if batch_size > len(self.memory):
            raise IndexError('Too few memory (%d) to draw a batch of %d' % (len(self.memory), batch_size))

        self.batches_drawn += 1

        if self.prioritized_replay:
            return self.draw_prioritized_batch(batch_size)
        else:
            return random.sample(self.memory, batch_size)

    # The nature paper doesn't do this but they mention the idea. see the results without this and remove
    def draw_prioritized_batch(self, batch_size):
        batch = []
        probe = MemoryType(None, 0, 0, None, False)
        while len(batch) < batch_size:
            probe.cumulative_weight = random.uniform(0, self.memory[-1].cumulative_weight)
            index = bisect.bisect_right(self.memory, probe, 0, len(self.memory) - 1)
            sample = self.memory[index]
            sample.weight = max(1, .8 * sample.weight)
            if sample not in batch:
                batch.append(sample)

        if self.batches_drawn % 100 == 0:
            cumulative = 0
            for sample in self.memory:
                cumulative += sample.weight
                sample.cumulative_weight = cumulative
        return batch

In [ ]:
def huber_loss_custom(y_actual, y_predicted):
    difference = tf.keras.backend.abs(y_actual - y_predicted)
    quadratic_part = tf.keras.backend.clip(difference, 0.0, 1.0)
    linear_part = difference - quadratic_part
    errors = (0.5 * tf.keras.backend.square(quadratic_part)) + linear_part
    loss = tf.keras.backend.sum(errors)
    return loss


class SingleBiasLayer(tf.keras.layers.Layer):
    def __init__(self, units, *args, **kwargs):
        super(SingleBiasLayer, self).__init__(*args, **kwargs)
        self.units = units

    def build(self, units):
        self.bias = self.add_weight('bias',
                                    shape=[self.units],
                                    initializer='zeros',
                                    trainable=True)
        super(SingleBiasLayer, self).build(units)  # Be sure to call this somewhere!

    def call(self, x):
        return tf.add(x ,self.bias)

    def get_config(self):
        config = {'units': self.units}
        base_config = super(SingleBiasLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class DQN:
    def __init__(self, action_size, out_dir, args):

        self.out_dir = out_dir
        self.target_model_update_freq = args.target_model_update_freq
        self.gamma = args.gamma
        self.learning_rate = args.learning_rate

        self.epoch_total_qval = 0
        self.epoch_train_step = 0
        self.epoch_loss = 0


        print("Building network for Policy Model")
        self.policy_model = self.buildNetwork()
        print("Building network for Target Model")
        self.target_model = self.buildNetwork()

        self.target_model.set_weights(self.policy_model.get_weights())

    def buildNetwork(self):

        frames_input = tf.keras.Input(shape=(84,84,4), name='frames')
        actions_input = tf.keras.Input(shape=(6,), name='filter')
        conv_1 = layers.Conv2D(32, (8, 8), strides=(4,4), input_shape= (84,84,4),
                                activation = 'relu', padding= 'valid',
                                data_format='channels_last', use_bias=True)(tf.keras.layers.Lambda(lambda x: x /255.0)(frames_input))
        conv_2 = layers.Conv2D(64, (4, 4), strides=(2,2), input_shape= (84,84,4),
                                activation = 'relu', padding= 'valid',
                                data_format='channels_last', use_bias=True)(conv_1)
        conv_3 = layers.Conv2D(64, (3, 3), strides=(1,1), input_shape= (84,84,4),
                                activation = 'relu', padding= 'valid',
                                data_format='channels_last', use_bias=True)(conv_2)
        conv_flattened = layers.Flatten()(conv_3)
        hidden = layers.Dense(512, activation ='relu', use_bias=True)(conv_flattened)
        output_wo_bias = layers.Dense(6, activation='linear', use_bias=False)(hidden)
        output = SingleBiasLayer(units=1)(output_wo_bias)
        filtered_output = layers.multiply([output, actions_input])
        model = models.Model(inputs = [frames_input, actions_input], outputs = filtered_output)
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate,rho=0.95,epsilon=0.01), loss='mse', metrics=["accuracy"])
        model.summary()
        return model

    def predict(self, screens, action_mask = None):
        if action_mask is None:
            action_mask = np.array([1,1,1,1,1,1]).reshape(1,6)
        q_values = self.policy_model.predict([screens,action_mask], batch_size = 1)
        q_values = np.squeeze(q_values)
        return np.argmax(q_values)

    def train(self, minibatch, step_number):

        next_screens_arr = [obj.next_state.get_screens()[0] for obj in minibatch]
        action_mask_target = np.ones((len(minibatch), 6))
        next_state_qval_arr_t = self.target_model.predict_on_batch([np.array(next_screens_arr),action_mask_target])
        next_state_qval_arr_p = self.policy_model.predict_on_batch([np.array(next_screens_arr),action_mask_target])

        current_screens_arr = [obj.current_state.get_screens()[0] for obj in minibatch]

        action_mask_policy = np.zeros((len(minibatch), 6))
        current_state_qval_arr = np.zeros((len(minibatch),6))

        for i in range(0, len(minibatch)):
            action_mask_policy[i][minibatch[i].action] = 1
            if minibatch[i].terminal:
                current_state_qval_arr[i][minibatch[i].action] = minibatch[i].reward
            else:
                current_state_qval_arr[i][minibatch[i].action] = minibatch[i].reward + self.gamma * next_state_qval_arr_t[i][np.argmax(next_state_qval_arr_p[i])]

        self.hist = self.policy_model.fit([np.asarray(current_screens_arr),action_mask_policy], current_state_qval_arr, batch_size = len(minibatch), verbose=0)

        self.epoch_total_qval += np.sum(current_state_qval_arr)
        self.epoch_train_step += 1
        self.epoch_loss += self.hist.history['loss'][0]

        if step_number % self.target_model_update_freq == 0:
            self.target_model.set_weights(self.policy_model.get_weights())

    def save_model(self,name="model.h5"):
        self.policy_model.save(self.out_dir + "/" +name)

In [ ]:
class GameTrainer:
    def __init__(self,out_dir):
        self.out_dir = out_dir
        self.args = Parameters()
        self.epoch_steps = self.args.epoch_steps
        self.agent = GameAgent(self.args)
        self.dqn = DQN(self.agent.action_size, self.out_dir, self.args)
        self.storage = StorageMemory(self.args)

        self.eval_epoch_num = 0
        self.train_epoch_num = 0
        self.train_game_count = 0
        self.stats = []


    def update_epsilon(self):
        self.epsilon = max(0.01, (1.0 - 0.9 * self.agent.step_number / 1000000))

    def train_epoch(self, num_epochs, eval_num_games = 100):
        for i in range(num_epochs):
            self.train_epoch_num += 1

            step_start = self.agent.step_number
            epoch_score_total = 0
            epoch_game_count = 0

            while((self.agent.step_number - step_start) < self.epoch_steps ):
                state = None
                game_score = 0
                no_op_action = 0
                no_op_max = random.randint(0,30)

                while (not self.agent.game_done):
                    self.update_epsilon()

                    if state is None or random.random() > (1 - self.epsilon):
                        action =  random.randrange(self.agent.action_size)
                    else:
                        screens = state.get_screens()
                        action = self.dqn.predict(screens)

                    if no_op_action < no_op_max:
                        action = 0
                        no_op_action += 1

                    old_state = state
                    reward, state, terminal = self.agent.take_action(action)

                    game_score += reward

                    if old_state is not None:
                        clipped_reward = min(1, max(-1,reward))
                        # clipped_reward = reward
                        self.storage.add_memory(MemoryType(old_state, action, clipped_reward, state, terminal))

                        if (self.agent.step_number > self.args.observation_steps) and (self.agent.episode_step % 4 == 0): #maybe delete %4
                            minibatch = self.storage.draw_batch(64) #WANDB
                            self.dqn.train(minibatch, self.agent.step_number)

                    if terminal:
                        state = None
                self.agent.reset_game()
                epoch_score_total += game_score
                epoch_game_count += 1
                self.train_game_count += 1

                self.stats.append([self.agent.step_number, game_score])


            with open(self.out_dir + '/training.csv', mode='a') as self.training_csv:
                self.training_writer = csv.writer(self.training_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                for log_ind in range(len(self.stats)):
                    self.training_writer.writerow(self.stats[log_ind])
                self.training_csv.close()

            with open(self.out_dir + '/training_epoch.csv', mode='a') as self.training_epoch_csv:
                self.training_epoch_writer = csv.writer(self.training_epoch_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                self.training_epoch_writer.writerow([self.train_epoch_num, epoch_game_count, epoch_score_total/epoch_game_count])
                self.training_epoch_csv.close()
            #WANDB
            #WANDB
            wandb.log({'epoch_score': epoch_score_total/epoch_game_count})
            wandb.log({'epoch_game_count': epoch_game_count})
            wandb.log({'epoch_loss_total': self.dqn.epoch_loss})
            wandb.log({'epoch_avg_qval': self.dqn.epoch_total_qval/self.dqn.epoch_train_step})
            self.dqn.epoch_total_qval = 0
            self.dqn.epoch_train_step = 0
            self.dqn.epoch_loss = 0


            self.stats = []
            self.agent.epoch_number += 1
            # self.eval_epoch(eval_num_games)

    def eval_epoch(self,num_games):
        play_agent = GameAgent(self.args)
        self.eval_epoch_num += 1
        total_score = 0
        for game in range(num_games):
            play_agent.reset_game()
            done = False
            life = 0
            game_score = 0
            state = None
            while not done:
                if(random.random() > 0 and state is not None):
                    screens = state.get_screens()
                    action = self.dqn.predict(screens)
                else:
                    action = random.randrange(play_agent.action_size)

                reward, state, terminal = play_agent.take_action(action)

                life += terminal
                game_score += reward
                if(life == 3):
                    done = True
            total_score += game_score

        average_score = total_score / num_games
        with open(self.out_dir + '/evaluation_epoch.csv', mode='a') as self.evaluation_epoch_csv:
            self.evaluation_epoch_writer = csv.writer(self.evaluation_epoch_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            self.evaluation_epoch_writer.writerow([self.eval_epoch_num, average_score])
            self.evaluation_epoch_csv.close()

In [ ]:
class Parameters:
    def __init__(self):
        self.replay_capacity = 1000000 #state to store for training
        self.target_model_update_freq =  10000 #steps to update target model
        self.learning_rate = 0.00025 #learning rate
        self.gamma = 0.99 #gamma for Q learning
        self.observation_steps = 70 #training starts after observation_steps
        self.prioritized_replay = False #Prioritize critical states when training (e.g. terminal or non zero rewards)")
        self.epoch_steps = 100000 #Number of steps to perform in an epoch



In [ ]:
cd '/content/drive/My Drive/WANDB'

/content/drive/My Drive/WANDB


In [ ]:
!wandb login xxxxxxxxxxxxxxxxxxxx
id = "config_7"
wandb.init(project="space-invaders-dueling-network", id=id, resume="allow")

wandb.config.batch_size = 64
wandb.config.learning_rate = 0.00025
wandb.config.optimizer = 'RMS'
wandb.config.taget_update_freq = 10000
wandb.config.loss = 'MSE'


# agent.save(os.path.join(wandb.run.dir, "model.h5"))
# wandb.log({'cumulative_avg_reward': cumulative_avg_reward})

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


Streaming file created twice in same run: /content/drive/My Drive/WANDB/wandb/run-20200429_233024-config_7/wandb-events.jsonl
Streaming file created twice in same run: /content/drive/My Drive/WANDB/wandb/run-20200429_233024-config_7/wandb-history.jsonl


In [ ]:
baseOutputDir = 'config_7'
if not os.path.isdir(baseOutputDir):
    os.makedirs(baseOutputDir)

In [ ]:
#Create the trainer
trainer = GameTrainer(baseOutputDir)

Building network for Policy Model
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
frames (InputLayer)             [(None, 84, 84, 4)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 84, 84, 4)    0           frames[0][0]                     
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 20, 20, 32)   8224        lambda[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 9, 64)     32832       conv2d[0][0]                     
____________________________________________________________

In [ ]:
if(os.path.exists(baseOutputDir + '/GameAgent_obj')):
    with open(baseOutputDir + '/GameAgent_obj', 'rb') as input:
        trainer.agent = pickle.load(input)
    with open(baseOutputDir + '/Storage_obj', 'rb') as input:
        trainer.storage = pickle.load(input)
    trainer.dqn.policy_model.load_weights(baseOutputDir + '/model_' + str(trainer.agent.epoch_number) + '.h5')
    trainer.dqn.target_model.set_weights(trainer.dqn.policy_model.get_weights())
    # with open(baseOutputDir +'/optimizer.pkl', 'rb') as f:
    #     weight_values = pickle.load(f)
    #     trainer.dqn.policy_model.optimizer.set_weights(weight_values)
    print("Done loading prev checkpoint")
else:
    print('No files found')

Done loading prev checkpoint


In [ ]:
#Start the training, specify the number of epochs
while True:
    trainer.train_epoch(1, 100)
    trainer.dqn.save_model('model_'+ str(trainer.agent.epoch_number) + '.h5')
    trainer.dqn.save_model('model.h5')
    with open(baseOutputDir + '/GameAgent_obj', 'wb') as output:
        pickle.dump(trainer.agent, output, pickle.HIGHEST_PROTOCOL)
    with open(baseOutputDir + '/Storage_obj', 'wb') as output:
        pickle.dump(trainer.storage, output, pickle.HIGHEST_PROTOCOL)

    # symbolic_weights = getattr(trainer.dqn.policy_model.optimizer, 'weights')
    # weight_values = tf.keras.backend.batch_get_value(symbolic_weights)
    # with open(baseOutputDir +'/optimizer.pkl', 'wb') as f:
    #     pickle.dump(weight_values, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
